In [1]:
import torch
import torch.nn as nn
import numpy as np
import json

In [2]:
def LoadData ( ) :
    global TRAIN_DATA, TEST_DATA, SEQ_LENGTH, ID_TO_EMOTION, EMOTION_TO_ID
    global ID_TO_WORD, WORD_TO_ID, VOCAB

    with open('DATA/TRAIN.json', 'r') as file :
        TRAIN_DATA = json.load(file)
    with open('DATA/TEST.json', 'r') as file :
        TEST_DATA = json.load(file)

    SEQ_LENGTH = 0
    for val in TRAIN_DATA.values() :
        for seq in val :
            if len(seq) > SEQ_LENGTH :
                SEQ_LENGTH = len(seq)
    for val in TEST_DATA.values() :
        for seq in val :
            if len(seq) > SEQ_LENGTH :
                SEQ_LENGTH = len(seq)
    
    ID_TO_EMOTION = dict(enumerate(list(TRAIN_DATA.keys())))
    EMOTION_TO_ID = { v : k for k , v in ID_TO_EMOTION.items() }

    VOCAB = set()
    for sentences in TRAIN_DATA.values() :
        for sentence in sentences :
            VOCAB.update(sentence)
    for sentences in TEST_DATA.values() :
        for sentence in sentences :
            VOCAB.update(sentence)

    ID_TO_WORD = list(VOCAB)
    ID_TO_WORD.sort()
    ID_TO_WORD.insert(0, '')
    WORD_TO_ID = { word : idd for idd, word in enumerate(ID_TO_WORD) }

In [3]:
class EmotionClassifier ( nn.Module ) :
    def __init__ ( self , input_size, hidden_size, output_size, num_layers ) :
        super(EmotionClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.embed = nn.Embedding(len(VOCAB), input_size)
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, 
                            num_layers = num_layers, dropout = 0.5, batch_first = True)
        self.dense = nn.Linear(hidden_size, output_size)
    
    def forward ( self , inputs ) :
        orig_inputs = inputs
        inputs = self.embed(inputs)
        
        h0 = torch.zeros(self.num_layers, inputs.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, inputs.size(0), self.hidden_size)
        output, _ = self.lstm(inputs, (h0, c0))
        
        batch_size = inputs.shape[0]
        masking_indices = [ np.argmax(orig_inputs[x] == 0) - 1 for x in range(batch_size) ]
        output = torch.stack( [ output[idd, k, :] for idd, k in enumerate(masking_indices) ] )
        
        categs = self.dense(output)
        return categs

In [4]:
def PredictEmotion ( sentence ) :
    sentence = sentence.lower()
    permitted_syms = set('abcdefghijklmnopqrstuvwxyz ')
    invalid_syms = set(sentence) - permitted_syms
    for sym in invalid_syms :
        sentence = sentence.replace(sym, ' ')
    
    words = sentence.split(' ')
    words = [ w for w in words if w != '' ]
    for word in words :
        if not word in ID_TO_WORD :
            print('[SENTENCE CONTAINS A WORD <{}> NOT PRESENT IN THE VOCABULARY]'.format(word))
            return
    
    encoded = []
    for word in words :
        encoded.append(WORD_TO_ID[word])
    
    label_dists = MODEL(torch.tensor([encoded]).long())
    pred = torch.argmax(label_dists, dim = 1).item()
    return ID_TO_EMOTION[pred]

In [5]:
LoadData()
MODEL = torch.load('MODELS/sentiment_analyser')
MODEL.eval()

EmotionClassifier(
  (embed): Embedding(9918, 128)
  (lstm): LSTM(128, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dense): Linear(in_features=256, out_features=5, bias=True)
)

In [6]:
PredictEmotion("Sometimes I feel very lonely.")

'sad'

In [7]:
PredictEmotion("How dare you touch my stuff without asking me!")

'anger'

In [8]:
PredictEmotion("These are those activities that really help my mood.")

'joy'

In [9]:
PredictEmotion("What is the date today?")

'neutral'

In [10]:
PredictEmotion("I do not want to lose my job.")

'fear'

In [11]:
PredictEmotion("I just can't tolerate this student's presence in my class.")

'anger'

In [12]:
PredictEmotion("You can't because we do not have enough money for this trip.")

'anger'

In [13]:
PredictEmotion("I am hardly able to focus on my studies these days.")

'sad'

In [14]:
PredictEmotion("Today I met him after such a long time.")

'joy'

In [15]:
PredictEmotion("Well, it is not the first time I have won this competition.")

'neutral'

In [16]:
PredictEmotion("I worked so hard for the promotion but still did not succeed.")

'sad'

In [17]:
PredictEmotion("I am too scared to go for swimming in that cursed pool.")

'fear'

In [18]:
PredictEmotion("I would never be able to jump that far.")

'neutral'

In [19]:
PredictEmotion("I was standing in the middle of nowhere and then this man started following me.")

'fear'

In [20]:
PredictEmotion("You did not have the right to say this about me.")

'neutral'

In [21]:
PredictEmotion("I just can't stand lazy people.")

'anger'

In [22]:
PredictEmotion("I should not take sweets from strangers.")

'sad'

In [23]:
PredictEmotion("We could not go for the vacation because of this appointment.")

'anger'